In [1]:
from cartopy import config
import cartopy.crs as ccrs
import matplotlib.colors as plt_cols

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time, json, requests, pprint, copy, math, dateutil
from datetime import datetime, timedelta
from argovisHelpers import helpers as avh

import os

from Argovis_tasks_helpers import get_route

/Users/dgiglio/opt/miniconda3/envs/argovis_demos/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
API_KEY=''

### This will be updated with an API query
Argovis supports several different data sets with the API and data structures described here. They and their corresponding routes are:

 - Argo profiling float data, `/argo`
 - CCHDO ship-based profile data, `/cchdo`
 - tropical cyclone data from HURDAT and JTWC, `/tc`
 - Global Drifter Program data, `/drifters`
 - Easy Ocean, `/easyocean`
 - several gridded products:
   - Roemmich-Gilson total temperature and salinity, `/grids/rg09`
   - ocean heat content, `/grids/kg21`
   - GLODAP, `/grids/glodap`
 - Argone Argo float position forecast model data, `/argone`
 - Argo trajectory data, `/argotrajectories`
 - several satellite-based timeseries:
   - NOAA sea surface temperature, `/timeseries/noaasst`
   - Copernicus sea surface height, `/timeseries/copernicussla`
   - CCMP wind vector product, `/timeseries/ccmpwind`

In [3]:
# API call showing collections available for each route

#### parameters the user needs to select
# user selects list of collections
collections = ['argo', 'cchdo', 'tc', 'extended/ar', 'drifters', 
               'easyocean', 'grids/rg09', '/grids/kg21',
               '/grids/glodap', '/argotrajectories', 
               '/timeseries/noaasst', '/timeseries/copernicussla',
               '/timeseries/ccmpwind'
              ]

# time period of interest for the API query
startDate = '2015-01-01T00:00:00Z'
endDate   = '2015-04-01T00:00:00Z'

# region to query could be added here

# long, lat grid for the output map
dx = 1
dy = 1
x_edges = np.arange(-180,181,dx)
y_edges = np.arange(-90,91,dy)

# verticalRange will be used only if it makes sense for that collection
verticalRange = '4000,4100'

In [4]:
def map_count_in_bins(name,x_edges,y_edges,dx,dy,fpath,startDate,endDate,vmin_map=1,vmax_map=1000):
    d = globals()[name] 
    lons = d['longitudes']#[x[1] for x in d]
    lats  = d['latitudes']#[x[2] for x in d]
    datetimes  = d['datetimes'] #[datetime.strptime(x[3][0:18]+'Z',"%Y-%m-%dT%H:%M:%SZ") for x in d]
    
    fig = plt.figure(figsize=(21, 7))
    
    h = np.histogram2d(lons, lats,[x_edges, y_edges])
    ax = plt.axes(projection=ccrs.PlateCarree())

    plt.pcolormesh(x_edges[0:-1]+dx/2, y_edges[0:-1]+dy/2, h[0].transpose(),  #60,
             transform=ccrs.PlateCarree(),norm=plt_cols.LogNorm(vmin=vmin_map, vmax=vmax_map ))

    ax.stock_img()
    ax.coastlines()
    
    plt.colorbar()

    plt.savefig(fpath+name+startDate[0:10]+'_'+endDate[0:10]+'.png')
    plt.show()

In [5]:
def create_df_from_minimal_query(mydict_name,api_output):
    bfr = pd.DataFrame(globals()[api_output])
    
    bfr.rename( columns={0 :'id'}, inplace=True )
    bfr.rename( columns={1 :'longitudes'}, inplace=True )
    bfr.rename( columns={2 :'latitudes'}, inplace=True )
    bfr.rename( columns={3 :'datetimes'}, inplace=True )
    bfr.rename( columns={4 :'source'}, inplace=True )
    return bfr

In [6]:
# define params for the query
params = {
        'startDate': startDate,
        'endDate': endDate,
        'compression': 'minimal'
    }

In [8]:
# for each collection of interest, plot a map
for icollection in collections:
    print(icollection)
    iroute = get_route(icollection)
    print(iroute)
    iparams = params
    #iparams['presRange'] = verticalRange
    globals()[icollection] = avh.query(icollection, options=iparams, apikey=API_KEY, apiroot=iroute)
    
    # create dataframe with columns of interest
    mydict_name = 'df_'+icollection
    globals()[mydict_name] = create_df_from_minimal_query(mydict_name=mydict_name,api_output=icollection)
    print(globals()[mydict_name])

argo
https://argovis-api.colorado.edu/
              id  longitudes  latitudes                 datetimes  \
0   5902341_333D -175.695010 -35.087770  2015-01-29T08:32:06.431Z   
1   5902342_131D -175.105250 -34.685310  2015-01-27T15:14:36.959Z   
2   5902341_332D -175.577000 -35.119530  2015-01-23T02:04:10.272Z   
3   5902342_130D -175.128580 -34.606100  2015-01-21T08:48:44.352Z   
4   5902341_331D -175.428020 -35.175740  2015-01-16T19:14:12.192Z   
5    7900660_005  115.148000 -61.053000  2015-01-16T07:43:43.000Z   
6    6901632_112  -21.725382  46.260268  2015-01-16T07:30:00.000Z   
7   5902342_129D -175.300200 -34.686760  2015-01-15T01:26:36.960Z   
8   5902341_330D -175.324310 -35.118280  2015-01-10T11:56:55.104Z   
9   5902342_128D -175.550260 -34.626940  2015-01-08T19:12:38.880Z   
10   7900660_004  112.754000 -60.641000  2015-01-06T07:29:35.000Z   
11  5902341_329D -175.335320 -34.998770  2015-01-04T04:31:16.896Z   
12   7900658_017  158.936000 -59.417000  2015-01-03T09:44:18.000

Exception: 400